![Image of Yaktocat](http://yuml.me/c961b9c3)

#States
* INIT
  * reads underlined text
  * extracts context
    * set next state = PREAMBLE
    
* PREAMBLE
  * reads PREAMBLE (NOTICE IS HEREBY...)
  * extracts preamble
    * set next state = PROCESS_ADVERTS

* PROCESS_ADVERTS
  * reads first key and first chunk of value
    * set next state = PROCESS_ADVERT
    
* PROCESS_ADVERT
  * reads next chunk and evaluate.
    ```
    if ':' in chunk then 
       store current key/value
       set current key, and initial value
    else
       append chunk to current-value
    ```



In [1]:
from bs4 import BeautifulSoup as Soup
from IPython.display import HTML
import pandas as pd
import numpy as np
import re
import pprint
from IPython.display import display

In [2]:
class State:
    def run(self):
        # implement this for printing trace statements
        assert 0, "implement me"
        
    def next(self, input):
        assert 0, "implement me"
        
    def flush(self, record):
        raise Exception('wtf!')



In [3]:
class Adverts(State):
    
    def __init__(self):
        State.__init__(self)
        
        # Current key and Value
        # value can span multiple lines/paragraphs
        # so we have to build it up a chunk at a time.
        # The complete text of value is assumed when
        # a new key/value pair is detected
        self.key = None
        self.value = None
        
        # Current Advertisement
        self.advert = {}
        self.processingAdvert = False
        
    def run(self):
        print('Process ADVERTS')
        
    def flush(self, record):
        self.advert[self.key] = self.value.strip()
        record['adverts'].append(self.advert)
        
    def next(self, para, record):
        text = para.get_text(strip=True)
        if ':' in text:
            k, v = text.split(':', 1)
            k = k.strip()
            if k.lower().startswith('agency'):
                # the first time the code gets here
                # processingAdvert is False, set it to True
                # so that from now on getting here means
                # we've completed an Advertisement
                
                if self.processingAdvert:
                    # End of record reached
                    self.advert[self.key] = self.value.strip()
                    record['adverts'].append(self.advert)
                    self.advert = {}
                
                # yep, processing advertisements
                self.processingAdvert = True
                    
            else:
                # A new key/value pair is detected,
                # store current key/value pair
                self.advert[self.key] = self.value.strip()
                
            # Initialize the current key/value pair
            self.key, self.value = k, v
            
        else:
            # A value can span multiple lines
            # keep adding chunks to the current value
            # until a new key/value pair is detected...
            if self.value:
                self.value += text   # Append to a previous value
            else:
                self.value = text    # Otherwise set value
            
        return self
        

class Preamble(State):
    def run(self):
        print('PREAMBLE')
        
    def next(self, para, record):
        text = para.get_text(strip=True).strip()
        if 'NOTICE IS HEREBY' in text:
            record['preamble'] = text
            return Adverts()
        return self
            

class Init(State):
    def run(self):
        print('INIT')
        
    def next(self, para, record):
        if para.find_all('u'):
            record['context'] = para.get_text(strip=True).strip()
            return Preamble()
        return self

        

In [4]:
def parse_notice(soup):
    state = Init()
    record = { 'adverts': []}

    for para in soup.find_all('p'):
    #     state.run()
    #     print (para)
        state = state.next(para, record)
    #     print ('\n\n')

    state.flush(record)
    return record



In [5]:
fn = 'orig.procPublicationRequest.oct-dec-2014.csv'
rows = pd.read_csv(fn, header=0, sep='|', encoding='latin-1')

In [6]:
cols = ['RequestID', 'StartDate', 'EndDate',
        'AgencyCode', 'AgencyName', 'AgencyDivision',
        'TypeOfNoticeCode', 'TypeOfNoticeDescription',
        'ShortTitle', 'SectionID', 'SectionName',
        'DueDate', 'ConfirmationNumber',
        'AdditionalDescription']

In [7]:
rows = rows[cols]

In [8]:
# from IPython.display import display
# badcount = 0
# for html in rows.AdditionalDescription.values:
#     if not isinstance(html, str):
#         badcount += 1
#     else:
#         display(HTML(html))
#         try:
#             parse_notice(Soup(html))
#         except Exception:
#             badcount += 1

In [9]:
import json
def scrape(row):
    output = None
    try:
        output = parse_notice(Soup(row.AdditionalDescription))
    except Exception :
        output = {'error' : 'bad input: [{}]'.format(row.AdditionalDescription)}
    row['output'] = json.dumps(output)
    return row

In [10]:
mocs = rows['AgencyName'] == "Mayor's Office of Contract Services"
meets = rows['TypeOfNoticeDescription'] == "Notice"
fix = rows[mocs & meets].apply(scrape,1)


In [11]:
for rec in fix[['output', 'AdditionalDescription']].values:
    output, text = rec
    if not isinstance(text, str):
        text = 'NAN'
    display(HTML(text))
    print('---')
    pprint.pprint(json.loads(output))
    display(HTML('<hr />'))
    print('\n\n')

---
{'adverts': [{'Agency': 'Department of Information Technology & '
                        'Telecommunications',
              'Award method of original contract': 'Intergovernmental',
              'Description of services': 'Design, development and '
                                         'deployment of application '
                                         'enhancements and extensions to '
                                         'the existing APT system along '
                                         'with the appropriate '
                                         'documentation required.',
              'End date of original contract': '1/31/2015',
              'FMS contract type': 'Consultant',
              'Headcount of personnel in substantially similar titles within agency': '4',
              'Method of renewal/extension the agency intends to utilize': 'Extension',
              'Modifications sought to the nature of services performed under the contract': 'None',
   

---
{'adverts': [{'Agency': 'Department of Environmental Protection',
              'Description of services sought': 'MTR-101: Provide all '
                                                'necessary labor, '
                                                'materials and equipment '
                                                'for the service and '
                                                'repair of various '
                                                'electric motors located '
                                                'at various Pumping '
                                                'Stations and associated '
                                                'New York City Department '
                                                'of Environmental '
                                                'Protection (DEP) '
                                                'facilities.',
              'End date of the proposed contract': 'April 30, 2017',
           

---
{'adverts': [{'Agency': 'DDC',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of extension the agency intends to utilize': 'Amendment '
                                                                   'Extension',
              'Modifications sought to the nature of services performed under the contract': 'No',
              'Nature of services': 'Requirements Contract for CM '
                                    'Services, Citywide for Transportation '
                                    'Unit',
              'New end date of the proposed extended contract': '6/29/2016',
              'New start date of the proposed extended contract': '6/30/2015',
              'Personnel in substantially similar titles within agency': 'none',
              'Reason(s) the agency intends to extend the contract': 'Continuity '
                                                                     'of '
                               

---
{'adverts': [{'Agency': 'DDC',
              'Description of services sought': 'A&E Requirements Contract '
                                                'for Commissioning',
              'End date of the proposed contract': '8/11/2018',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of solicitation the agency intends to utilize': 'Request '
                                                                      'for '
                                                                      'Proposal',
              'Personnel in substantially similar titles within agency': 'none',
              'Start date of the proposed contract': '8/12/2015'},
             {'Agency': 'DDC',
              'Description of services sought': 'Requirements Contract for '
                                                'HVAC/Fire '
                                                'Protection/Electrical/Plumbing, '
                        

---
{'adverts': [{'Agency': 'DDC',
              'Description of services sought': 'Queensboro Hall Municipal '
                                                'Parking Garage – '
                                                'Demolition',
              'End date of the proposed contract': '1/15/2018',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of solicitation the agency intends to utilize': 'Task '
                                                                      'Order',
              'Personnel in substantially similar titles within agency': 'None',
              'Start date of the proposed contract': '1/15/2015'}],
 'context': 'Notice of Intent to Issue New Solicitation(s) Not Included in '
            'FY 2015 Annual Contracting Plan and Schedule',
 'preamble': 'NOTICE IS HEREBY GIVENthat the Mayor will be issuing the '
             'following solicitation(s) not included in the FY 2015 Annual '
          

...

---
{'adverts': [{'Agency': 'Department of Information Technology',
              'Description of services sought': 'Technical Architect to '
                                                'provide System '
                                                'Architecture Review and '
                                                'Planning for NYC Business',
              'End date of the proposed contract': '1/31/16',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of solicitation the agency intends to utilize': 'Task '
                                                                      'Order',
              'Personnel in substantially similar titles within agency': 'None',
              'Start date of the proposed contract': '2/1/15'},
             {'Agency': 'Department of Information Technology',
              'Description of services sought': 'Content Management System '
                                            

---
{'adverts': [{'Agency': 'Department of Parks and Recreation',
              'End date of the proposed contract': '5/31/2020',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of solicitation the agency intends to utilize': 'Sole '
                                                                      'Source',
              'Nature of services sought': 'Upgrade of Software for '
                                           'Recreation membership system '
                                           'to a web-based system',
              'Personnel in substantially similar titles within agency': 'None',
              'Start date of the proposed contract': '6/01/2015'}],
 'context': 'Notice of Intent to Issue New Solicitation(s) Not Included in '
            'FY 2015 Annual Contracting Plan and Schedule',
 'preamble': 'NOTICE IS HEREBY GIVENthat the Mayor will be issuing the '
             'following solicitation(s) not inclu

---
{'adverts': [{'Agency': 'Department of Finance',
              'Description of services sought': 'On-Site Mobile Shredding '
                                                'Services',
              'End date of the proposed contract': '6/30/2020',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of solicitation the agency intends to utilize': 'Required '
                                                                      'Method '
                                                                      '(Preferred '
                                                                      'Source)',
              'Personnel in substantially similar titles within agency': 'None',
              'Start date of the proposed contract': '7/1/2015'}],
 'context': 'Notice of Intent to Issue New Solicitation(s) Not Included in '
            'FY 2015 Annual Contracting Plan and Schedule',
 'preamble': 'NOTICE IS HEREBY GIVENthat the

---
{'adverts': [{'Agency': 'Department of Health and Mental HygieneNature of '
                        'services sought: Provision of public education '
                        'messages through media campaigns servicesStart '
                        'date of the proposed contract: 12/01/2015End date '
                        'of the proposed contract: 11/30/2018Method of '
                        'solicitation the agency intends to utilize: '
                        'Competitive Sealed ProposalPersonnel in '
                        'substantially similar titles within agency: '
                        'NoneHeadcount of personnel in substantially '
                        'similar titles within agency: 0'}],
 'context': 'Notice of Intent to Issue New Solicitation(s) Not Included in '
            'FY 2015 Annual Contracting Plan and Schedule',
 'preamble': 'NOTICE IS HEREBY GIVENthat the Mayor will be issuing the '
             'following solicitation(s) not included in the FY 2015 Ann

---
{'adverts': [{'Agency': 'Department of Environmental Protection',
              'Description of services sought': 'SEQNS-FLD DSGN: Design '
                                                'and Design during '
                                                'Construction for SE '
                                                'Queens Flooding '
                                                'Mitigation Program.',
              'End date of the proposed contract': '1/14/2020',
              'Headcount of personnel in substantially similar titles within agency': '0',
              'Method of solicitation the agency intends to utilize': 'Request '
                                                                      'for '
                                                                      'Proposals',
              'Personnel in substantially similar titles within agency': 'None',
              'Start date of the proposed contract': '1/15/2016'}],
 'context': 'Notice of Intent to 